In [3]:
import pandas as pd
import glob
import os

In [10]:
os.chdir("/Users/pranavtavildar/Desktop/Senior-Thesis/data")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames = all_filenames[:12]
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "ChevronOil2022.csv", index=False, encoding='utf-8-sig')
combined_csv["Text"] =combined_csv["Text"].map(str)

In [11]:
from transformers import pipeline

In [12]:
sentiment_classifier = pipeline('sentiment-analysis')

combined_csv = (combined_csv.assign(sentiment = lambda x: x['Text'].apply(lambda s: sentiment_classifier(s)))
.assign(
label = lambda x: x['sentiment'].apply(lambda s: (s[0]['label'])),
score = lambda x: x['sentiment'].apply(lambda s: (s[0]['score']))))
combined_csv.head()

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceCla

,Date,Tweet ID,Text,Username,Likes,sentiment,label,score,Open,High,Low,Close,Adj Close,Volume
0,2022-06-01 23:59:03+00:00,1532149762392309761,@Chevron Don’t let gov tell you what to do as ...,RobertA15891551,3.0,"[{'label': 'NEGATIVE', 'score': 0.992789328098...",NEGATIVE,0.992789,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-06-01 23:44:08+00:00,1532146007219195904,@RepStefanik No they are the direct result of ...,keithlb,0.0,"[{'label': 'NEGATIVE', 'score': 0.990190863609...",NEGATIVE,0.990191,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-01 23:40:44+00:00,1532145150750867457,"@Jim_Jordan Tell Exon, Shell, Texaco, Chevron,...",atxsouthside04,0.0,"[{'label': 'NEGATIVE', 'score': 0.994984269142...",NEGATIVE,0.994984,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-06-01 23:36:52+00:00,1532144177374408709,"When Chevron's PAC is maxed out to you, don't ...",EvanMMinton,12.0,"[{'label': 'NEGATIVE', 'score': 0.996644496917...",NEGATIVE,0.996644,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-06-01 23:18:44+00:00,1532139614734934017,$BRK.A $BRK.B $BRK OIL GAS ENERGY BOOM \nWhile...,Icon00007,0.0,"[{'label': 'POSITIVE', 'score': 0.769509613513...",POSITIVE,0.769510,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
combined_csv.to_csv("../data/sentiment_scored_chevron.csv",encoding='utf-8', index=False)